# Import Libraries and files 

In [1]:
# Basic Libraries
import os
import pandas as pd
import numpy as np



# Pre-processing
import fitz # install PyMuPDF
import requests
from tqdm import tqdm
from spacy.lang.en import English 
import re

# Embeddings
from sentence_transformers import util, SentenceTransformer

# RAG
import torch
import textwrap
from time import perf_counter as timer

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# HuggingFace token
access_token = os.environ['hugging_face_api_key']

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 
from transformers import BitsAndBytesConfig
from transformers import pipeline

In [2]:
# Get PDF document path
filename = "human-nutrition-text.pdf"

# Download PDF
if not os.path.exists(filename):
    print("File doesn't exist, downloading...")
    
    # The URL of the PDF you want to download
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # Send a GET request to the url
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the file and save it
        with open(filename,"wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been download and saved as {filename}")
    else: print(f"[INFO] Failed to download the file. Status Code: {response.status_code}")
        
else:
  print(f"File {filename} exists.")

File human-nutrition-text.pdf exists.


# Read Files

In [3]:
# Format text read from PDF
def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n"," ").strip()
    return cleaned_text

# Provide pdf name
def open_and_read_pdf(filename: str)->list[dict]:
    doc = fitz.open(filename)
    pages_and_texts=[]
    for page_number,page in tqdm(enumerate(doc)):
        text = page.get_text()
        formatted_text = text_formatter(text)

        # store pages information and texts in a dictionary
        pages_and_texts.append({"page_number": page_number - 41,  # adjust page numbers since our PDF starts on page 42
                        "page_char_count": len(text),
                        "page_word_count": len(text.split(" ")),
                        "page_sentence_count_raw": len(text.split(". ")),
                        # "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                        "text": text})
        
    return pages_and_texts

def read_pages(pages,page_number:int):
    # insert pages_and_text and page number to retrieve the specified page in the pdf
    content  = pages[page_number+41]
    return content

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Text Processing

In [4]:
def text_processor(pages_and_texts,num_sentence_chunk_size=10,min_token_length=30):
    nlp = English()
    # Add a sentencizer pipeline
    nlp.add_pipe("sentencizer")

    # Sentenciaing texts on each page to sentences
    for item in tqdm(pages_and_texts):
        item["sentences"] = list(nlp(item["text"]).sents)
        
        # Make sure all sentences are strings
        item["sentences"] = [str(sentence) for sentence in item["sentences"]]
        
        # Count the sentences 
        item["page_sentence_count_spacy"] = len(item["sentences"])

    # Loop through pages and texts and split sentences into chunks
    for item in tqdm(pages_and_texts):
        item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                            slice_size=num_sentence_chunk_size)
        item["num_chunks"] = len(item["sentence_chunks"])

    # Split each chunk into its own item
    pages_and_chunks = []
    for item in tqdm(pages_and_texts):
        for sentence_chunk in item["sentence_chunks"]:
            chunk_dict = {}
            chunk_dict["page_number"] = item["page_number"]
            
            # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
            joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
            chunk_dict["sentence_chunk"] = joined_sentence_chunk

            # Get stats about the chunk
            chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
            chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
            chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
            
            pages_and_chunks.append(chunk_dict)

    df = pd.DataFrame(pages_and_chunks)
    pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
    return pages_and_chunks, pages_and_chunks_over_min_token_len

# Embedding

In [5]:
def embedding(texts,model_name="all-mpnet-base-v2",device="cuda"):

    # use "all-mpnet-base-v2"
    embedding_model = SentenceTransformer(model_name_or_path=model_name, 
                                        device=device)
    # # Send the model to the GPU
    # embedding_model.to("cuda") 

    # Create embeddings one by one on the GPU
    # pages_and_chunks_over_min_token_len
    for item in tqdm(texts):
        item["embedding"] = embedding_model.encode(item["sentence_chunk"])

    # # Turn text chunks into a single list
    # text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

    # # Embed all texts in batches (z-book: 34.4s)
    # text_chunk_embeddings = embedding_model.encode(text_chunks,
    #                                                batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
    #                                                convert_to_tensor=True) # optional to return embeddings as tensor instead of array

    # Save embeddings to file
    text_chunks_and_embeddings_df = pd.DataFrame(texts)
    embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
    text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)
    return text_chunks_and_embeddings_df

In [6]:
# Define embedding model
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device='cuda')

In [7]:
def read_embedding_from_file(file='text_chunks_and_embeddings_df.csv',device='cuda'):
    # Import texts and embedding df
    text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")
    # Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
    text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

    # Convert texts and embedding df to list of dicts
    pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

    # Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
    embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
    return pages_and_chunks,embeddings

# Retrieval

In [8]:
# Pre-processing
pages_and_texts = open_and_read_pdf("human-nutrition-text.pdf")
pages_and_chunks,pages_and_chunks_over_min_token_len = text_processor(pages_and_texts)
read_pages(pages_and_chunks_over_min_token_len,258)

1208it [00:00, 1275.33it/s]
100%|██████████| 1208/1208 [00:00<00:00, 37750.77it/s]


{'page_number': 196,
 'sentence_chunk': 'Potassium also is involved in protein synthesis, energy metabolism, \nand platelet function, and acts as a buffer in blood, playing a role in \nacid-base balance.\nImbalances of Potassium \nInsufficient potassium levels in the body (hypokalemia) can be \ncaused by a low dietary intake of potassium or by high sodium \nintakes, but more commonly it results from medications that \nincrease water excretion, mainly diuretics. The signs and symptoms \nof hypokalemia are related to the functions of potassium in nerve \ncells and consequently skeletal and smooth-muscle contraction.\nThe signs and symptoms include muscle weakness and cramps, \nrespiratory distress, and constipation. Severe potassium depletion \ncan cause the heart to have abnormal contractions and can even \nbe fatal. High levels of potassium in the blood, or hyperkalemia, \nalso affects the heart. It is a silent condition as it often displays \nno signs or symptoms. Extremely high level

In [9]:
# Read embedding from csv file
pages_and_chunks,embeddings = read_embedding_from_file()

In [10]:
# Define helper function to print wrapped text 
def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [11]:
def dot_product(vector1, vector2):
    return torch.dot(vector1, vector2)

def cosine_similarity(vector1, vector2):
    dot_product = torch.dot(vector1, vector2)

    # Get Euclidean/L2 norm of each vector (removes the magnitude, keeps direction)
    norm_vector1 = torch.sqrt(torch.sum(vector1**2))
    norm_vector2 = torch.sqrt(torch.sum(vector2**2))

    return dot_product / (norm_vector1 * norm_vector2)

In [12]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Pass user query into the function,
    Embeds a query with model and returns top k (default=5) scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    # Get dot product scores based on embeddings passed into the function
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    """
    Return top 5 results based on dot product scores    
    """
    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    print("Results:")
    # Loop through zipped together scores and indicies
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        # Print the page number too so we can reference the textbook further and check the results
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")

# Generation

In [13]:
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 10 GB


In [14]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False 
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 10 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [15]:
# 1. Create quantization config for smaller model loading (optional)
# Requires !pip install bitsandbytes accelerate, see: https://github.com/TimDettmers/bitsandbytes, https://huggingface.co/docs/accelerate/
# For models that require 4-bit quantization (use this if you have low GPU memory available)

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

# Bonus: Setup Flash Attention 2 for faster inference, default to "sdpa" or "scaled dot product attention" if it's not available
# Flash Attention 2 requires NVIDIA GPU compute capability of 8.0 or above, see: https://developer.nvidia.com/cuda-gpus
# Requires !pip install flash-attn, see: https://github.com/Dao-AILab/flash-attention 
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 2. Pick a model we'd like to use (this will depend on how much GPU memory you have available)
#model_id = "google/gemma-7b-it"
model_id = model_id # (we already set this above)
print(f"[INFO] Using model_id: {model_id}")

# 3. Instantiate tokenizer (tokenizer turns text into numbers ready for the model) 
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id, token=access_token)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
                                                 torch_dtype=torch.float16, # datatype to use, we want float16
                                                 quantization_config=quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage=False, # use full memory 
                                                 attn_implementation=attn_implementation) # which attention version to use


if not use_quantization_config: # quantization takes care of device setting automatically, so if it's not used, send model to GPU 
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: google/gemma-2b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Augmented Generation

In [16]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    
    context_items are a subset of pages_and_chunks
    By matching idices from the top 5 results
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # Create a base prompt with examples to help the model
    # Note: this is very customizable, I've chosen to use 3 examples of the answer style we'd like.
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""

    # Update base prompt with context items and query   
    base_prompt = base_prompt.format(context=context, query=query)

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [17]:
def ask(query, 
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    
    # Get just the scores and indices of top related results
    # query: user query
    # embeddings: documents embeddings
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)
    
    # Create a list of context items
    # context_items are top 5 results we get from indices
    context_items = [pages_and_chunks[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU 
        
    # Format the prompt with context items
    # Pass user query and top 5 results into the prompt
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    
    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    # Only return the answer without the context items
    if return_answer_only:
        return output_text
    
    return output_text, context_items

# Final
Using our pipeline

In [18]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management."
]

# Manually created question list
manual_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "water soluble vitamins"
]

query_list = gpt4_questions + manual_questions

In [ ]:
query = query_list[0]
print(f"Query: {query}\n")

# Answer query with context and return context 
answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print("#"*50,"\n")
print(f"Answer:\n")
print_wrapped(answer)
print("#"*50,"\n")
print(f"Context items:")
context_items

Query: What are the macronutrients, and what roles do they play in the human body?

[INFO] Time taken to get scores on 1683 embeddings: 0.00013 seconds.


c:\Users\cheefoo3\OneDrive\Code\Python\DeepLearning.AI\14 local-RAG-Daniel_Bourke\simple-local-rag\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:561: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


################################################## 

Answer:

Sure. Here's the answer to the user's query:  The macronutrients are
carbohydrates, lipids, and proteins. These nutrients provide energy for the body
and contribute to the construction and repair of tissues. Proteins are essential
for tissue formation, cell repair, and hormone and enzyme production.
Carbohydrates provide a ready source of energy for the body and provide
structural constituents for cell formation. Lipids provide stored energy for the
body, function as structural components of cells, and aid in the transmission of
signals.
################################################## 

Context items:


[{'page_number': 5,
  'sentence_chunk': 'Macronutrients \nNutrients \nthat \nare \nneeded \nin \nlarge \namounts \nare \ncalled \nmacronutrients. There are three classes of macronutrients: \ncarbohydrates, lipids, and proteins. These can be metabolically \nprocessed into cellular energy. The energy from macronutrients \ncomes from their chemical bonds. This chemical energy is \nconverted into cellular energy that is then utilized to perform work, \nallowing our bodies to conduct their basic functions. A unit of \nmeasurement of food energy is the calorie. On nutrition food labels \nthe amount given for “calories” is actually equivalent to each calorie \nmultiplied by one thousand. A kilocalorie (one thousand calories, \ndenoted with a small “c”) is synonymous with the “Calorie” (with a \ncapital “C”) on nutrition food labels. Water is also a macronutrient in \nthe sense that you require a large amount of it, but unlike the other \nmacronutrients, it does not yield calories.\nCarbohydra